In [64]:
import pandas as pd
Partner_data = pd.read_excel('Case study - Partner Data.xlsx', sheet_name='Sheet1')

In [65]:
Partner_data.head()

,Partner ID,Date,Supply Hours,Tasks
0,233711,2019-04-04,3.336667,4
1,233711,2019-04-05,10.195000,8
2,238310,2019-04-25,3.051944,6
3,238310,2019-04-26,6.639722,8
4,238310,2019-04-27,11.263889,14


In [66]:
Partner_data.nunique()

Partner ID       1524
Date              120
Supply Hours    25174
Tasks              41
dtype: int64

In [67]:
Partner_data.isnull().sum()

Partner ID      0
Date            0
Supply Hours    0
Tasks           0
dtype: int64

In [ ]:
#Data preprocessing
#Convert date column to date format

In [68]:
Partner_data['Date'] = pd.to_datetime(Partner_data['Date'])

In [ ]:
# Calculate the total supply hours per partner per day

In [69]:
Partner_data['Total Supply Hours'] = Partner_data.groupby(['Partner ID', 'Date'])['Supply Hours'].transform('sum')

In [70]:
Partner_data.head()

,Partner ID,Date,Supply Hours,Tasks,Total Supply Hours
0,233711,2019-04-04,3.336667,4,3.336667
1,233711,2019-04-05,10.195000,8,10.195000
2,238310,2019-04-25,3.051944,6,3.051944
3,238310,2019-04-26,6.639722,8,6.639722
4,238310,2019-04-27,11.263889,14,11.263889


In [9]:
# Calculate the total tasks per partner per day

In [71]:
Partner_data['Total Tasks'] = Partner_data.groupby(['Partner ID', 'Date'])['Tasks'].transform('sum')

In [11]:
Partner_data.head()

,Partner ID,Date,Supply Hours,Tasks,Total Supply Hours,Total Tasks
0,233711,2019-04-04,3.336667,4,3.336667,4
1,233711,2019-04-05,10.195000,8,10.195000,8
2,238310,2019-04-25,3.051944,6,3.051944,6
3,238310,2019-04-26,6.639722,8,6.639722,8
4,238310,2019-04-27,11.263889,14,11.263889,14


In [ ]:
# Calculate the day of the week for each date
# Monday: 0, Sunday: 6

In [72]:
Partner_data['Day of Week'] = Partner_data['Date'].dt.dayofweek

In [73]:
Partner_data.head()

,Partner ID,Date,Supply Hours,Tasks,Total Supply Hours,Total Tasks,Day of Week
0,233711,2019-04-04,3.336667,4,3.336667,4,3
1,233711,2019-04-05,10.195000,8,10.195000,8,4
2,238310,2019-04-25,3.051944,6,3.051944,6,3
3,238310,2019-04-26,6.639722,8,6.639722,8,4
4,238310,2019-04-27,11.263889,14,11.263889,14,5


In [ ]:
# Proposed Point Structure

In [75]:
Partner_data['Base Points'] = Partner_data['Total Supply Hours']
Partner_data['Weekly Bonus'] = ((Partner_data.groupby('Partner ID')['Day of Week'].transform(lambda x: set(x) == set([0, 1, 2, 3, 4, 5, 6]))) & (Partner_data['Total Supply Hours'] >= 4)).astype(int) * 50
Partner_data['Weekend Bonus'] = ((Partner_data.groupby('Partner ID')['Day of Week'].transform(lambda x: set(x) == set([6, 0]))) & (Partner_data['Total Supply Hours'] >= 8)).astype(int) * 100
Partner_data['Total Points'] = Partner_data['Base Points'] + Partner_data['Weekly Bonus'] + Partner_data['Weekend Bonus']

In [76]:
Partner_data.head()

,Partner ID,Date,Supply Hours,Tasks,Total Supply Hours,Total Tasks,Day of Week,Base Points,Weekly Bonus,Weekend Bonus,Total Points
0,233711,2019-04-04,3.336667,4,3.336667,4,3,3.336667,0,0,3.336667
1,233711,2019-04-05,10.195000,8,10.195000,8,4,10.195000,0,0,10.195000
2,238310,2019-04-25,3.051944,6,3.051944,6,3,3.051944,0,0,3.051944
3,238310,2019-04-26,6.639722,8,6.639722,8,4,6.639722,0,0,6.639722
4,238310,2019-04-27,11.263889,14,11.263889,14,5,11.263889,0,0,11.263889


In [ ]:
#Budget summary

In [77]:
budget_summary = Partner_data.groupby('Partner ID').agg({
    'Total Tasks': 'sum',
    'Total Points': 'sum'
}).reset_index()

In [ ]:
#Total Cost Per Point
#Determine CPP value that keeps CPT value below 0.9

In [78]:
CPP = 0.9 / budget_summary['Total Tasks'].sum() 

In [79]:
CPP

2.1299506561431326e-06

In [ ]:
# Calculate the cost of points earned and CPT impact

In [80]:
budget_summary['Cost of Points Earned'] = budget_summary['Total Points'] * CPP
budget_summary['CPT Impact'] = (budget_summary['Cost of Points Earned'] / budget_summary['Total Tasks']).round(6)

In [81]:
budget_summary

,Partner ID,Total Tasks,Total Points,Cost of Points Earned,CPT Impact
0,233711,12,13.531667,0.000029,0.000002
1,238310,57,38.803056,0.000083,0.000001
2,240387,71,30.004444,0.000064,0.000001
3,240582,404,1596.296667,0.003400,0.000008
4,240673,1,0.501389,0.000001,0.000001
...,...,...,...,...,...
1519,2760517,14,10.533611,0.000022,0.000002
1520,2762480,8,11.023611,0.000023,0.000003
1521,2763021,5,8.118889,0.000017,0.000003
1522,2763210,12,12.590833,0.000027,0.000002


In [ ]:
# Print the proposed point structure and budget summary

In [82]:
# Print the proposed point structure and budget summary
print("Proposed Point Structure:")
print(Partner_data[['Partner ID', 'Date', 'Base Points', 'Weekly Bonus', 'Weekend Bonus', 'Total Points']])
print("\nBudget Summary:")
print(budget_summary[['Partner ID', 'Total Tasks', 'Cost of Points Earned', 'CPT Impact']])

Proposed Point Structure:
       Partner ID       Date  Base Points  Weekly Bonus  Weekend Bonus  \
0          233711 2019-04-04     3.336667             0              0   
1          233711 2019-04-05    10.195000             0              0   
2          238310 2019-04-25     3.051944             0              0   
3          238310 2019-04-26     6.639722             0              0   
4          238310 2019-04-27    11.263889             0              0   
...           ...        ...          ...           ...            ...   
36308     2760517 2019-04-30    10.533611             0              0   
36309     2762480 2019-04-30    11.023611             0              0   
36310     2763021 2019-04-30     8.118889             0              0   
36311     2763210 2019-04-30    12.590833             0              0   
36312     2771806 2019-04-30     0.587500             0              0   

       Total Points  
0          3.336667  
1         10.195000  
2          3.051944

In [29]:
Partner_data.head(10)

,Partner ID,Date,Supply Hours,Tasks,Total Supply Hours,Total Tasks,Day of Week,Base Points,Weekly Bonus,Weekend Bonus,Total Points
0,233711,2019-04-04,3.336667,4,3.336667,4,3,3.336667,0,0,3.336667
1,233711,2019-04-05,10.195000,8,10.195000,8,4,10.195000,0,0,10.195000
2,238310,2019-04-25,3.051944,6,3.051944,6,3,3.051944,0,0,3.051944
3,238310,2019-04-26,6.639722,8,6.639722,8,4,6.639722,0,0,6.639722
4,238310,2019-04-27,11.263889,14,11.263889,14,5,11.263889,0,0,11.263889
5,238310,2019-04-28,8.488889,16,8.488889,16,6,8.488889,0,0,8.488889
6,238310,2019-04-29,9.358611,13,9.358611,13,0,9.358611,0,0,9.358611
7,240387,2019-04-25,5.368889,11,5.368889,11,3,5.368889,0,0,5.368889
8,240387,2019-04-26,8.275556,20,8.275556,20,4,8.275556,0,0,8.275556
9,240387,2019-04-27,8.083889,20,8.083889,20,5,8.083889,0,0,8.083889


In [83]:
Partner_data.to_excel('Proposed_Point_structure.xlsx', index=False)

In [84]:
budget_summary.head(10)

,Partner ID,Total Tasks,Total Points,Cost of Points Earned,CPT Impact
0,233711,12,13.531667,0.000029,0.000002
1,238310,57,38.803056,0.000083,0.000001
2,240387,71,30.004444,0.000064,0.000001
3,240582,404,1596.296667,0.003400,0.000008
4,240673,1,0.501389,0.000001,0.000001
5,240678,1,1.486389,0.000003,0.000003
6,240711,39,63.101667,0.000134,0.000003
7,240715,11,31.886667,0.000068,0.000006
8,240743,150,692.080000,0.001474,0.000010
9,240748,1,0.898611,0.000002,0.000002


In [85]:
budget_summary.to_excel('Budget_Summary.xlsx', index=False)

In [86]:
Partner_data['Date'] = pd.to_datetime(Partner_data['Date'])
# Extract the month and year from the 'Date' column
Partner_data['Month'] = Partner_data['Date'].dt.month
Partner_data['Year'] = Partner_data['Date'].dt.year

In [87]:
Partner_data.head()

,Partner ID,Date,Supply Hours,Tasks,Total Supply Hours,Total Tasks,Day of Week,Base Points,Weekly Bonus,Weekend Bonus,Total Points,Month,Year
0,233711,2019-04-04,3.336667,4,3.336667,4,3,3.336667,0,0,3.336667,4,2019
1,233711,2019-04-05,10.195000,8,10.195000,8,4,10.195000,0,0,10.195000,4,2019
2,238310,2019-04-25,3.051944,6,3.051944,6,3,3.051944,0,0,3.051944,4,2019
3,238310,2019-04-26,6.639722,8,6.639722,8,4,6.639722,0,0,6.639722,4,2019
4,238310,2019-04-27,11.263889,14,11.263889,14,5,11.263889,0,0,11.263889,4,2019


In [ ]:
# Calculate the total cost of points earned and total number of completed tasks for each month

In [88]:
monthly_summary = Partner_data.groupby(['Year', 'Month']).agg({'Total Points': 'sum', 'Total Tasks': 'sum'})

In [89]:
monthly_summary.head()

Total Points  Total Tasks
Year Month                            
2019 1      345339.084444        86825
     2      403475.213056       103865
     3      456438.477778       112178
     4      435361.436111       119677

In [ ]:
# Calculate the CPT Impact for each month

In [90]:
monthly_summary['CPT Impact'] = monthly_summary['Total Points'] / monthly_summary['Total Tasks']
monthly_summary['CPT Impact'] = monthly_summary['CPT Impact'].round(3)

In [91]:
monthly_summary.head()

Total Points  Total Tasks  CPT Impact
Year Month                                        
2019 1      345339.084444        86825       3.977
     2      403475.213056       103865       3.885
     3      456438.477778       112178       4.069
     4      435361.436111       119677       3.638

In [57]:
print(monthly_summary)

             Total Points  Total Tasks  CPT Impact
Year Month                                        
2019 1      345339.084444        86825       3.977
     2      403475.213056       103865       3.885
     3      456438.477778       112178       4.069
     4      435361.436111       119677       3.638


In [92]:
monthly_summary.to_excel('Monthly_wise_CPT.xlsx', index=False)